# Exploración de Descarga de Imágenes Sentinel-2

Este notebook demuestra cómo usar las funciones de `src/` para descargar
y visualizar imágenes Sentinel-2.

**Autor:** Equipo 24  
**Fecha:** Noviembre 2025

In [ ]:
# Imports
# NO necesitamos sys.path.append porque poetry hace src/ importable
from src.utils.sentinel_download import (
    create_sentinel_config,
    download_sentinel2_bands
)
from src.utils.image_processing import (
    create_rgb_image,
    create_false_color_image
)
from src.features.ndvi_calculator import calculate_ndvi

import matplotlib.pyplot as plt
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

print("✅ Imports completados")

In [ ]:
# Configuración
client_id = os.getenv('SENTINEL_HUB_CLIENT_ID')
client_secret = os.getenv('SENTINEL_HUB_CLIENT_SECRET')

config = create_sentinel_config(client_id, client_secret)
print("✅ Sentinel Hub configurado")

In [ ]:
# Definir área de interés (Lima, Perú)
bbox = {
    'min_lat': -12.1,
    'min_lon': -77.1,
    'max_lat': -12.0,
    'max_lon': -77.0
}
print(f"Área de interés: {bbox}")

In [ ]:
# Descargar datos
print("Descargando datos de Sentinel-2...")
data = download_sentinel2_bands(
    bbox_coords=bbox,
    config=config,
    date_from='2024-01-01',
    date_to='2024-01-31'
)
print(f"✅ Descarga completada: {data['metadata']['dimensions']}")
print(f"Período: {data['metadata']['date_from']} a {data['metadata']['date_to']}")

In [ ]:
# Visualizar RGB
bands = data['bands']
rgb = create_rgb_image(bands['B04'], bands['B03'], bands['B02'])

plt.figure(figsize=(12, 10))
plt.imshow(rgb)
plt.title('Sentinel-2 RGB')
plt.axis('off')
plt.show()

In [ ]:
# Visualizar False Color (NIR-Red-Green)
false_color = create_false_color_image(bands['B08'], bands['B04'], bands['B03'])

plt.figure(figsize=(12, 10))
plt.imshow(false_color)
plt.title('Sentinel-2 False Color (NIR-R-G)')
plt.axis('off')
plt.show()

In [ ]:
# Calcular NDVI
ndvi_result = calculate_ndvi(bands['B04'], bands['B08'], bands['SCL'])
ndvi = ndvi_result['ndvi_masked']
stats = ndvi_result['statistics']

print("\nEstadísticas de NDVI:")
print(f"  Media: {stats['mean']:.3f}")
print(f"  Desviación estándar: {stats['std']:.3f}")
print(f"  Mínimo: {stats['min']:.3f}")
print(f"  Máximo: {stats['max']:.3f}")
print(f"  Cobertura de nubes: {stats['cloud_coverage']:.1f}%")

In [ ]:
# Visualizar NDVI
plt.figure(figsize=(12, 10))
plt.imshow(ndvi, cmap='RdYlGn', vmin=-0.2, vmax=0.8)
plt.colorbar(label='NDVI', shrink=0.8)
plt.title(f'NDVI (Media: {stats["mean"]:.3f})')
plt.axis('off')
plt.show()

In [ ]:
# Comparación lado a lado
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# RGB
axes[0].imshow(rgb)
axes[0].set_title('RGB')
axes[0].axis('off')

# False Color
axes[1].imshow(false_color)
axes[1].set_title('False Color (NIR-R-G)')
axes[1].axis('off')

# NDVI
im = axes[2].imshow(ndvi, cmap='RdYlGn', vmin=-0.2, vmax=0.8)
axes[2].set_title(f'NDVI (Media: {stats["mean"]:.3f})')
axes[2].axis('off')
plt.colorbar(im, ax=axes[2], shrink=0.8)

plt.tight_layout()
plt.show()